In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType


In [2]:
os.getcwd()

'/app/notebooks/01_eda'

In [2]:
# Initialize SparkSession with optimized settings for 32GB RAM
spark = pyspark.sql.SparkSession.builder \
    .appName("KKBox_Churn_Prediction") \
    .master("local[*]") \
    .config("spark.driver.memory", "24g") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

# Set log level to ERROR
spark.sparkContext.setLogLevel("ERROR")

# Verify configuration
print(f"Spark Driver Memory: {spark.conf.get('spark.driver.memory')}")
print(f"Spark Master: {spark.conf.get('spark.master')}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/07 11:32:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Driver Memory: 24g
Spark Master: local[*]


In [3]:
BRONZE_USER_LOGS = "/app/datamart_ori/bronze/user_logs"
BRONZE_TRANSACTIONS = "/app/datamart_ori/bronze/transactions"
BRONZE_MEMBERS = "/app/datamart/bronze/members"

BRONZE_USER_LOGS_OUTPUT = "/app/datamart/bronze/user_logs"
BRONZE_TRANSACTIONS_OUTPUT = "/app/datamart/bronze/transactions"

In [5]:
# ===============================
# 📥 Load Tables
# ===============================

print("Loading Bronze layer tables...\n")

df_userlogs = spark.read.parquet(BRONZE_USER_LOGS)
df_transactions = spark.read.parquet(BRONZE_TRANSACTIONS)
df_members = spark.read.parquet(BRONZE_MEMBERS)
    
print("Done\n")

Loading Bronze layer tables...



Done



In [6]:
# Make sure df_members has unique msno to avoid duplication
df_members_unique = df_members.select("msno").distinct()

# Filter userlogs
df_userlogs_filtered = (
    df_userlogs
        .join(df_members_unique, on="msno", how="inner")
)

# Filter transactions
df_transactions_filtered = (
    df_transactions
        .join(df_members_unique, on="msno", how="inner")
)


In [7]:
df_userlogs_filtered.write \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .parquet(BRONZE_USER_LOGS_OUTPUT)

In [8]:
df_transactions_filtered.write \
        .mode("overwrite") \
        .partitionBy("year", "month") \
        .parquet(BRONZE_TRANSACTIONS_OUTPUT)

In [10]:
RAW_DATA = "/app/data/"

In [11]:
df_userlogs_filtered.coalesce(2).write.csv(
    RAW_DATA,
    header=True,
    mode="overwrite"
)

df_transactions_filtered.coalesce(2).write.csv(
    RAW_DATA,
    header=True,
    mode="overwrite"
)

In [6]:
RAW_DATA = "/app/data/userlogs/"

In [5]:
df_userlogs_filtered = spark.read.parquet(BRONZE_USER_LOGS_OUTPUT)

In [7]:
df_userlogs_filtered.coalesce(2).write.csv(
    RAW_DATA,
    header=True,
    mode="overwrite"
)